<a href="https://colab.research.google.com/github/GabiHert/T2-BD2/blob/main/T2_BD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BANCO DE DADOS 2 - TRABALHO 2
## Sistema de Venda de Livros

---

# A2. IMPLEMENTAÇÃO DO BANCO DE DADOS NÃO-RELACIONAL

## Configurações de Acesso ao MongoDB Atlas

## Quando usar Embedding vs Referência

**Embedding (dados embutidos):**
- Arrays pequenos que sempre são consultados junto com o documento pai
- Dados que não precisam ser consultados separadamente
- Exemplo: endereços dos clientes, autores dos livros

**Referência:**
- Quando os dados são compartilhados entre vários documentos
- Quando preciso consultar esses dados independentemente
- Evita duplicação excessiva de informação
- Exemplo: categorias, clientes em pedidos

### Coleção: Pedidos

**Estrutura escolhida:**
- Itens do pedido ficam embutidos (array `itens[]`)
- Endereço de entrega também fica embutido (objeto `enderecoEntrega`)
- Cliente é uma referência (`Id_Cliente`)

**Por que fiz assim:**

Os itens ficam embutidos porque:
- Faz sentido, né? Um pedido sem itens não existe
- Sempre que vou consultar um pedido, preciso ver os itens
- Deixa tudo em um documento só, mais fácil de trabalhar

O endereço de entrega é um snapshot porque:
- Precisa ficar fixo depois que o pedido é feito
- Se o cliente mudar de endereço no cadastro, não pode alterar pedidos antigos
- É importante pra histórico e rastreamento

O cliente é referência porque:
- Evita repetir os dados do cliente em todo pedido
- Um cliente faz vários pedidos (relacionamento 1:N)

### Coleção: Produtos

**Estrutura escolhida:**
- Autores ficam embutidos (array `autor[]`)
- Categoria é referência (`Id_Categoria`)

**Por que fiz assim:**

Autores embutidos:
- Sempre consulto os autores junto com o livro
- A maioria dos livros tem poucos autores (1 a 3)
- Não preciso consultar autores separadamente

Categoria como referência:
- Tem poucas categorias mas muitos produtos
- Facilita fazer agrupamentos tipo "quantos livros por categoria"
- Se mudar o nome da categoria, mudo só em um lugar

### Coleção: Categorias

**Estrutura:** Coleção separada

**Por que separada:**
- São poucas categorias mas vários produtos
- Vai que eu precise mudar o nome de uma categoria, não quero ter que atualizar todos os produtos
- Facilita fazer consultas agrupadas por categoria

### Coleção: Clientes

**Estrutura escolhida:**
- Endereços ficam embutidos (array `enderecos[]`)

**Por que:**
- Sempre que consulto um cliente, quero ver os endereços dele
- Cada cliente tem poucos endereços (geralmente 1 ou 2)
- Não preciso consultar endereços sozinhos
- Fica mais rápido, pega tudo numa consulta só

### Diagrama do Modelo

![Esquema MongoDB](https://github.com/GabiHert/T2-BD2/blob/main/WhatsApp%20Image%202025-11-02%20at%2016.47.11.jpeg?raw=1)

# A1. MODELO NÃO-RELACIONAL

Aqui está o modelo não-relacional que desenvolvi para o sistema de venda de livros. Usei estratégias de embedding e referências conforme a necessidade.

# TRABALHO 2 - BANCO DE DADOS 2

**Tema:** Sistema de Venda de Livros

**Alunos:**
- Gabriel Guinter Herter
- Gabriel Kirchmann Kondach

##CONFIGURAÇÕES

### Instalação do PyMongo

In [ ]:
!pip install pymongo[srv,tls]

### Importação de bibliotecas do Python

In [ ]:
import pymongo
import json
import pandas as pd
from bson.json_util import dumps
from IPython.display import display
from pprint import pprint

### Configurando as credenciais de acesso e conectando ao BD



In [ ]:
mongo_usuario = "mongo" # insira o nome do seu usuário de acesso ao MongoDB Atlas [verifique em Database > Connnect > Drivers ]
mongo_senha = "mongo"   # insira sua senha para o usuário utilizado
mongo_host = "clusterbd2.88ps3qi.mongodb.net"   # verifique qual o host no MongoDB Atlas [verifique em Database Acess]

In [ ]:
conexao = pymongo.MongoClient("mongodb+srv://" + mongo_usuario + ":" + mongo_senha + "@" + mongo_host + "/?retryWrites=true&w=majority&appName=ClusterBD2")
print(conexao.name)

In [ ]:
conexao.list_database_names()

### Selecionando banco de dados

O banco de dados utilizado será **venda_livros**, que contém as coleções relacionadas ao sistema de venda de livros.

In [ ]:
bd = conexao["venda_livros"]

In [ ]:
bd.list_collection_names()

## INSERÇÃO DE DADOS

Vou criar as 4 coleções e inserir alguns dados de exemplo pra testar.

**Coleções:**
1. Categorias - tipos de livros
2. Produtos - os livros com seus autores
3. Clientes - clientes e endereços
4. Pedidos - pedidos com itens e endereço de entrega

### Categorias

Aqui ficam as categorias dos livros. É uma coleção separada que os produtos vão referenciar.

In [ ]:
bd.drop_collection("categorias")

In [ ]:
bd.categorias.insert_many([
    {
        "_id": 1,
        "nome": "Ficção",
        "descricao": "Livros de ficção e literatura em geral"
    },
    {
        "_id": 2,
        "nome": "Romance",
        "descricao": "Livros de romance e relacionamentos"
    },
    {
        "_id": 3,
        "nome": "Técnico",
        "descricao": "Livros técnicos e de tecnologia"
    },
    {
        "_id": 4,
        "nome": "Autoajuda",
        "descricao": "Livros de desenvolvimento pessoal e autoajuda"
    },
    {
        "_id": 5,
        "nome": "História",
        "descricao": "Livros de história e biografias"
    },
    {
        "_id": 6,
        "nome": "Infantil",
        "descricao": "Livros infantis e juvenis"
    }
])

### Produtos

Coleção dos livros com validation schema pra garantir que os dados estejam corretos.

Cada produto tem um array de autores embutido e uma referência pra categoria.

In [ ]:
# Remover coleção se existir
bd.drop_collection("produtos")

# Criar coleção com validation schema
bd.create_collection("produtos", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["_id", "titulo", "isbn", "ano", "preco", "quantidadeEstoque", "autor", "Id_Categoria"],
        "properties": {
            "_id": {
                "bsonType": "objectId",
                "description": "ID do produto - obrigatório"
            },
            "titulo": {
                "bsonType": "string",
                "description": "Título do livro - obrigatório"
            },
            "isbn": {
                "bsonType": "string",
                "description": "ISBN do livro - obrigatório"
            },
            "ano": {
                "bsonType": "int",
                "minimum": 1000,
                "maximum": 2100,
                "description": "Ano de publicação - obrigatório"
            },
            "preco": {
                "bsonType": "double",
                "minimum": 0,
                "description": "Preço do livro - obrigatório"
            },
            "quantidadeEstoque": {
                "bsonType": "int",
                "minimum": 0,
                "description": "Quantidade em estoque - obrigatório"
            },
            "autor": {
                "bsonType": "array",
                "minItems": 1,
                "description": "Array de autores - obrigatório, mínimo 1",
                "items": {
                    "bsonType": "object",
                    "required": ["nome", "nacionalidade"],
                    "properties": {
                        "nome": {
                            "bsonType": "string",
                            "description": "Nome do autor"
                        },
                        "nacionalidade": {
                            "bsonType": "string",
                            "description": "Nacionalidade do autor"
                        }
                    }
                }
            },
            "Id_Categoria": {
                "bsonType": "int",
                "description": "Referência à categoria - obrigatório"
            }
        }
    }
})

print("Coleção 'produtos' criada com validation schema!")

In [ ]:
from bson.objectid import ObjectId

bd.produtos.insert_many([
    {
        "_id": ObjectId(),
        "titulo": "1984",
        "isbn": "978-0451524935",
        "ano": 1949,
        "preco": 45.90,
        "quantidadeEstoque": 15,
        "autor": [
            {
                "nome": "George Orwell",
                "nacionalidade": "Britânica"
            }
        ],
        "Id_Categoria": 1
    },
    {
        "_id": ObjectId(),
        "titulo": "O Senhor dos Anéis",
        "isbn": "978-8533613379",
        "ano": 1954,
        "preco": 89.90,
        "quantidadeEstoque": 8,
        "autor": [
            {
                "nome": "J.R.R. Tolkien",
                "nacionalidade": "Britânica"
            }
        ],
        "Id_Categoria": 1
    },
    {
        "_id": ObjectId(),
        "titulo": "Clean Code",
        "isbn": "978-0132350884",
        "ano": 2008,
        "preco": 95.00,
        "quantidadeEstoque": 20,
        "autor": [
            {
                "nome": "Robert C. Martin",
                "nacionalidade": "Americana"
            }
        ],
        "Id_Categoria": 3
    },
    {
        "_id": ObjectId(),
        "titulo": "Orgulho e Preconceito",
        "isbn": "978-8544001646",
        "ano": 1813,
        "preco": 35.90,
        "quantidadeEstoque": 12,
        "autor": [
            {
                "nome": "Jane Austen",
                "nacionalidade": "Britânica"
            }
        ],
        "Id_Categoria": 2
    },
    {
        "_id": ObjectId(),
        "titulo": "O Poder do Hábito",
        "isbn": "978-8539004119",
        "ano": 2012,
        "preco": 42.90,
        "quantidadeEstoque": 25,
        "autor": [
            {
                "nome": "Charles Duhigg",
                "nacionalidade": "Americana"
            }
        ],
        "Id_Categoria": 4
    },
    {
        "_id": ObjectId(),
        "titulo": "Sapiens",
        "isbn": "978-8525432629",
        "ano": 2011,
        "preco": 54.90,
        "quantidadeEstoque": 18,
        "autor": [
            {
                "nome": "Yuval Noah Harari",
                "nacionalidade": "Israelense"
            }
        ],
        "Id_Categoria": 5
    },
    {
        "_id": ObjectId(),
        "titulo": "Harry Potter e a Pedra Filosofal",
        "isbn": "978-8532530802",
        "ano": 1997,
        "preco": 39.90,
        "quantidadeEstoque": 30,
        "autor": [
            {
                "nome": "J.K. Rowling",
                "nacionalidade": "Britânica"
            }
        ],
        "Id_Categoria": 6
    },
    {
        "_id": ObjectId(),
        "titulo": "Design Patterns",
        "isbn": "978-0201633610",
        "ano": 1994,
        "preco": 120.00,
        "quantidadeEstoque": 10,
        "autor": [
            {
                "nome": "Erich Gamma",
                "nacionalidade": "Suíça"
            },
            {
                "nome": "Richard Helm",
                "nacionalidade": "Australiana"
            },
            {
                "nome": "Ralph Johnson",
                "nacionalidade": "Americana"
            },
            {
                "nome": "John Vlissides",
                "nacionalidade": "Americana"
            }
        ],
        "Id_Categoria": 3
    },
    {
        "_id": ObjectId(),
        "titulo": "Dom Casmurro",
        "isbn": "978-8594318602",
        "ano": 1899,
        "preco": 28.90,
        "quantidadeEstoque": 14,
        "autor": [
            {
                "nome": "Machado de Assis",
                "nacionalidade": "Brasileira"
            }
        ],
        "Id_Categoria": 1
    },
    {
        "_id": ObjectId(),
        "titulo": "O Pequeno Príncipe",
        "isbn": "978-8522008731",
        "ano": 1943,
        "preco": 24.90,
        "quantidadeEstoque": 40,
        "autor": [
            {
                "nome": "Antoine de Saint-Exupéry",
                "nacionalidade": "Francesa"
            }
        ],
        "Id_Categoria": 6
    }
])

### Clientes

Os clientes com seus endereços embutidos no documento.

In [ ]:
bd.drop_collection("clientes")

In [ ]:
bd.clientes.insert_many([
    {
        "_id": ObjectId(),
        "nome": "João Silva",
        "email": "joao.silva@email.com",
        "telefone": "51999887766",
        "endereco": [
            {
                "rua": "Rua das Flores",
                "numero": 123,
                "complemento": "Apto 101",
                "cidade": "Porto Alegre",
                "estado": "RS",
                "regiao": "Sul",
                "cep": "90000-000"
            }
        ]
    },
    {
        "_id": ObjectId(),
        "nome": "Maria Santos",
        "email": "maria.santos@email.com",
        "telefone": "11988776655",
        "endereco": [
            {
                "rua": "Avenida Paulista",
                "numero": 1000,
                "complemento": "",
                "cidade": "São Paulo",
                "estado": "SP",
                "regiao": "Sudeste",
                "cep": "01310-100"
            },
            {
                "rua": "Rua Augusta",
                "numero": 500,
                "complemento": "Casa",
                "cidade": "São Paulo",
                "estado": "SP",
                "regiao": "Sudeste",
                "cep": "01305-000"
            }
        ]
    },
    {
        "_id": ObjectId(),
        "nome": "Pedro Oliveira",
        "email": "pedro.oliveira@email.com",
        "telefone": "21987654321",
        "endereco": [
            {
                "rua": "Avenida Atlântica",
                "numero": 2000,
                "complemento": "Cobertura",
                "cidade": "Rio de Janeiro",
                "estado": "RJ",
                "regiao": "Sudeste",
                "cep": "22021-001"
            }
        ]
    },
    {
        "_id": ObjectId(),
        "nome": "Ana Costa",
        "email": "ana.costa@email.com",
        "telefone": "85976543210",
        "endereco": [
            {
                "rua": "Rua do Sol",
                "numero": 456,
                "complemento": "",
                "cidade": "Fortaleza",
                "estado": "CE",
                "regiao": "Nordeste",
                "cep": "60000-000"
            }
        ]
    },
    {
        "_id": ObjectId(),
        "nome": "Carlos Mendes",
        "email": "carlos.mendes@email.com",
        "telefone": "51998765432",
        "endereco": [
            {
                "rua": "Rua dos Andradas",
                "numero": 789,
                "complemento": "Sala 5",
                "cidade": "Porto Alegre",
                "estado": "RS",
                "regiao": "Sul",
                "cep": "90020-000"
            }
        ]
    },
    {
        "_id": ObjectId(),
        "nome": "Julia Ferreira",
        "email": "julia.ferreira@email.com",
        "telefone": "41987651234",
        "endereco": [
            {
                "rua": "Rua XV de Novembro",
                "numero": 321,
                "complemento": "",
                "cidade": "Curitiba",
                "estado": "PR",
                "regiao": "Sul",
                "cep": "80000-000"
            }
        ]
    },
    {
        "_id": ObjectId(),
        "nome": "Lucas Almeida",
        "email": "lucas.almeida@email.com",
        "telefone": "61986543210",
        "endereco": [
            {
                "rua": "Esplanada dos Ministérios",
                "numero": 100,
                "complemento": "Bloco A",
                "cidade": "Brasília",
                "estado": "DF",
                "regiao": "Centro-Oeste",
                "cep": "70000-000"
            }
        ]
    },
    {
        "_id": ObjectId(),
        "nome": "Fernanda Lima",
        "email": "fernanda.lima@email.com",
        "telefone": "71985432109",
        "endereco": [
            {
                "rua": "Avenida Oceânica",
                "numero": 555,
                "complemento": "",
                "cidade": "Salvador",
                "estado": "BA",
                "regiao": "Nordeste",
                "cep": "40000-000"
            }
        ]
    }
])

### Pedidos

Pedidos com itens embutidos (que referenciam produtos) e snapshot do endereço de entrega.

Tem validation schema também pra garantir a estrutura correta.

In [ ]:
# Remover coleção se existir
bd.drop_collection("pedidos")

# Criar coleção com validation schema
bd.create_collection("pedidos", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["_id", "Id_Cliente", "dataPedido", "dataEntrega", "status", "enderecoEntrega", "itens"],
        "properties": {
            "_id": {
                "bsonType": "objectId",
                "description": "ID do pedido - obrigatório"
            },
            "Id_Cliente": {
                "bsonType": "objectId",
                "description": "Referência ao cliente - obrigatório"
            },
            "dataPedido": {
                "bsonType": "date",
                "description": "Data do pedido - obrigatório"
            },
            "dataEntrega": {
                "bsonType": "date",
                "description": "Data de entrega - obrigatório"
            },
            "status": {
                "bsonType": "string",
                "enum": ["Pendente", "Processando", "Enviado", "Entregue", "Cancelado"],
                "description": "Status do pedido - obrigatório"
            },
            "enderecoEntrega": {
                "bsonType": "object",
                "required": ["rua", "numero", "cidade", "estado", "cep"],
                "description": "Snapshot do endereço de entrega - obrigatório",
                "properties": {
                    "rua": {"bsonType": "string"},
                    "numero": {"bsonType": "int"},
                    "complemento": {"bsonType": "string"},
                    "cidade": {"bsonType": "string"},
                    "estado": {"bsonType": "string"},
                    "regiao": {"bsonType": "string"},
                    "cep": {"bsonType": "string"}
                }
            },
            "itens": {
                "bsonType": "array",
                "minItems": 1,
                "description": "Array de itens do pedido - obrigatório, mínimo 1",
                "items": {
                    "bsonType": "object",
                    "required": ["Id_Produto", "quantidade", "precoUnitario"],
                    "properties": {
                        "Id_Produto": {
                            "bsonType": "objectId",
                            "description": "Referência ao produto"
                        },
                        "quantidade": {
                            "bsonType": "int",
                            "minimum": 1,
                            "description": "Quantidade do produto"
                        },
                        "precoUnitario": {
                            "bsonType": "double",
                            "minimum": 0,
                            "description": "Preço unitário no momento do pedido"
                        }
                    }
                }
            },
            "total": {
                "bsonType": "double",
                "minimum": 0,
                "description": "Valor total do pedido"
            }
        }
    }
})

print("Coleção 'pedidos' criada com validation schema!")

In [ ]:
# Obter IDs de produtos e clientes para criar pedidos
# Pegamos os primeiros produtos e clientes inseridos

produtos_list = list(bd.produtos.find().limit(10))
clientes_list = list(bd.clientes.find().limit(8))

print(f"Produtos encontrados: {len(produtos_list)}")
print(f"Clientes encontrados: {len(clientes_list)}")

# Exibir alguns produtos para referência
print("\nPrimeiros 3 produtos:")
for p in produtos_list[:3]:
    print(f"  - {p['titulo']}: {p['_id']}")

In [ ]:
from datetime import datetime, timedelta

# Criar pedidos usando IDs reais de clientes e produtos
bd.pedidos.insert_many([
    {
        "_id": ObjectId(),
        "Id_Cliente": clientes_list[0]['_id'],  # João Silva
        "dataPedido": datetime(2024, 1, 15),
        "dataEntrega": datetime(2024, 1, 20),
        "status": "Entregue",
        "enderecoEntrega": clientes_list[0]['endereco'][0],  # Snapshot do endereço
        "itens": [
            {
                "Id_Produto": produtos_list[0]['_id'],  # 1984
                "quantidade": 2,
                "precoUnitario": produtos_list[0]['preco']
            },
            {
                "Id_Produto": produtos_list[2]['_id'],  # Clean Code
                "quantidade": 1,
                "precoUnitario": produtos_list[2]['preco']
            }
        ],
        "total": (2 * produtos_list[0]['preco']) + (1 * produtos_list[2]['preco'])
    },
    {
        "_id": ObjectId(),
        "Id_Cliente": clientes_list[1]['_id'],  # Maria Santos
        "dataPedido": datetime(2024, 2, 10),
        "dataEntrega": datetime(2024, 2, 15),
        "status": "Entregue",
        "enderecoEntrega": clientes_list[1]['endereco'][0],
        "itens": [
            {
                "Id_Produto": produtos_list[1]['_id'],  # O Senhor dos Anéis
                "quantidade": 1,
                "precoUnitario": produtos_list[1]['preco']
            },
            {
                "Id_Produto": produtos_list[6]['_id'],  # Harry Potter
                "quantidade": 3,
                "precoUnitario": produtos_list[6]['preco']
            }
        ],
        "total": (1 * produtos_list[1]['preco']) + (3 * produtos_list[6]['preco'])
    },
    {
        "_id": ObjectId(),
        "Id_Cliente": clientes_list[2]['_id'],  # Pedro Oliveira
        "dataPedido": datetime(2024, 3, 5),
        "dataEntrega": datetime(2024, 3, 12),
        "status": "Entregue",
        "enderecoEntrega": clientes_list[2]['endereco'][0],
        "itens": [
            {
                "Id_Produto": produtos_list[5]['_id'],  # Sapiens
                "quantidade": 2,
                "precoUnitario": produtos_list[5]['preco']
            },
            {
                "Id_Produto": produtos_list[4]['_id'],  # O Poder do Hábito
                "quantidade": 1,
                "precoUnitario": produtos_list[4]['preco']
            }
        ],
        "total": (2 * produtos_list[5]['preco']) + (1 * produtos_list[4]['preco'])
    },
    {
        "_id": ObjectId(),
        "Id_Cliente": clientes_list[0]['_id'],  # João Silva (segundo pedido)
        "dataPedido": datetime(2024, 4, 20),
        "dataEntrega": datetime(2024, 4, 27),
        "status": "Enviado",
        "enderecoEntrega": clientes_list[0]['endereco'][0],
        "itens": [
            {
                "Id_Produto": produtos_list[7]['_id'],  # Design Patterns
                "quantidade": 1,
                "precoUnitario": produtos_list[7]['preco']
            },
            {
                "Id_Produto": produtos_list[2]['_id'],  # Clean Code
                "quantidade": 2,
                "precoUnitario": produtos_list[2]['preco']
            }
        ],
        "total": (1 * produtos_list[7]['preco']) + (2 * produtos_list[2]['preco'])
    },
    {
        "_id": ObjectId(),
        "Id_Cliente": clientes_list[3]['_id'],  # Ana Costa
        "dataPedido": datetime(2024, 5, 1),
        "dataEntrega": datetime(2024, 5, 8),
        "status": "Entregue",
        "enderecoEntrega": clientes_list[3]['endereco'][0],
        "itens": [
            {
                "Id_Produto": produtos_list[9]['_id'],  # O Pequeno Príncipe
                "quantidade": 5,
                "precoUnitario": produtos_list[9]['preco']
            }
        ],
        "total": 5 * produtos_list[9]['preco']
    },
    {
        "_id": ObjectId(),
        "Id_Cliente": clientes_list[4]['_id'],  # Carlos Mendes
        "dataPedido": datetime(2024, 5, 15),
        "dataEntrega": datetime(2024, 5, 22),
        "status": "Processando",
        "enderecoEntrega": clientes_list[4]['endereco'][0],
        "itens": [
            {
                "Id_Produto": produtos_list[3]['_id'],  # Orgulho e Preconceito
                "quantidade": 1,
                "precoUnitario": produtos_list[3]['preco']
            },
            {
                "Id_Produto": produtos_list[8]['_id'],  # Dom Casmurro
                "quantidade": 2,
                "precoUnitario": produtos_list[8]['preco']
            }
        ],
        "total": (1 * produtos_list[3]['preco']) + (2 * produtos_list[8]['preco'])
    },
    {
        "_id": ObjectId(),
        "Id_Cliente": clientes_list[1]['_id'],  # Maria Santos (segundo pedido)
        "dataPedido": datetime(2024, 6, 10),
        "dataEntrega": datetime(2024, 6, 17),
        "status": "Entregue",
        "enderecoEntrega": clientes_list[1]['endereco'][1],  # Segundo endereço
        "itens": [
            {
                "Id_Produto": produtos_list[0]['_id'],  # 1984
                "quantidade": 1,
                "precoUnitario": produtos_list[0]['preco']
            },
            {
                "Id_Produto": produtos_list[1]['_id'],  # O Senhor dos Anéis
                "quantidade": 1,
                "precoUnitario": produtos_list[1]['preco']
            },
            {
                "Id_Produto": produtos_list[8]['_id'],  # Dom Casmurro
                "quantidade": 1,
                "precoUnitario": produtos_list[8]['preco']
            }
        ],
        "total": produtos_list[0]['preco'] + produtos_list[1]['preco'] + produtos_list[8]['preco']
    }
])

print("Pedidos inseridos com sucesso!")

## Verificando a implementação

Vou checar se tudo foi criado certinho e mostrar alguns exemplos dos dados.

In [ ]:
# Listar todas as coleções criadas
print("=== COLEÇÕES NO BANCO venda_livros ===")
colecoes = bd.list_collection_names()
for col in colecoes:
    count = bd[col].count_documents({})
    print(f"  - {col}: {count} documentos")

print("\n=== EXEMPLO DE DADOS ===")

# Mostrar um exemplo de cada coleção
print("\n[Categoria]")
pprint(bd.categorias.find_one())

print("\n[Produto]")
produto_exemplo = bd.produtos.find_one()
if produto_exemplo:
    print(f"Título: {produto_exemplo['titulo']}")
    print(f"Autores: {[a['nome'] for a in produto_exemplo['autor']]}")
    print(f"Preço: R$ {produto_exemplo['preco']}")

print("\n[Cliente]")
cliente_exemplo = bd.clientes.find_one()
if cliente_exemplo:
    print(f"Nome: {cliente_exemplo['nome']}")
    print(f"Email: {cliente_exemplo['email']}")
    print(f"Endereços: {len(cliente_exemplo['endereco'])}")

print("\n[Pedido]")
pedido_exemplo = bd.pedidos.find_one()
if pedido_exemplo:
    print(f"Data: {pedido_exemplo['dataPedido']}")
    print(f"Status: {pedido_exemplo['status']}")
    print(f"Total de itens: {len(pedido_exemplo['itens'])}")
    print(f"Valor total: R$ {pedido_exemplo['total']:.2f}")

# A3. CONSULTAS

Aqui estão as consultas pedidas no trabalho, usando agregação do MongoDB.

In [ ]:
## Métodos Auxiliares para Exibição de Dados

def listJson(dados):
    """Exibe os dados em formato JSON"""
    for d in dados:
        print(d)

def listTable(dados):
    """Exibe os dados em formato de tabela usando pandas"""
    df = pd.DataFrame(dados)
    display(df)

## A3.a) Quantidade de produtos por categoria

Vou contar quantos produtos tem em cada categoria.

**Como fiz:**
- Agrupo os produtos por categoria
- Faço lookup pra pegar o nome da categoria
- Conto quantos tem em cada

In [ ]:
pipeline = [
    {
        "$group": {
            "_id": "$Id_Categoria",
            "quantidade": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "categorias",
            "localField": "_id",
            "foreignField": "_id",
            "as": "categoria"
        }
    },
    {
        "$unwind": "$categoria"
    },
    {
        "$project": {
            "_id": 0,
            "categoria": "$categoria.nome",
            "quantidade_produtos": "$quantidade"
        }
    },
    {
        "$sort": {"categoria": 1}
    }
]

resultado = list(bd.produtos.aggregate(pipeline))
print("=== QUANTIDADE DE PRODUTOS POR CATEGORIA ===")
listTable(resultado)

## A3.b) Entregas com endereço e nome do cliente

Lista todos os pedidos com os dados de endereço e o nome do cliente, ordenado por data de entrega.

**Como fiz:**
- Junto pedidos com clientes usando lookup
- Pego os dados do endereço de entrega (que já tá no pedido como snapshot)
- Ordeno por data

In [ ]:
pipeline = [
    {
        "$lookup": {
            "from": "clientes",
            "localField": "Id_Cliente",
            "foreignField": "_id",
            "as": "cliente"
        }
    },
    {
        "$unwind": "$cliente"
    },
    {
        "$project": {
            "_id": 0,
            "nome_cliente": "$cliente.nome",
            "data_entrega": "$dataEntrega",
            "rua": "$enderecoEntrega.rua",
            "numero": "$enderecoEntrega.numero",
            "cidade": "$enderecoEntrega.cidade",
            "estado": "$enderecoEntrega.estado",
            "cep": "$enderecoEntrega.cep",
            "status": "$status"
        }
    },
    {
        "$sort": {"data_entrega": 1}
    }
]

resultado = list(bd.pedidos.aggregate(pipeline))
print("=== ENTREGAS COM ENDEREÇO E CLIENTE ===")
listTable(resultado)

## A3.c) Pedidos com valor acima de R$ 1000

Mostrar só os pedidos que passam de mil reais.

**Como fiz:**
- Filtro pedidos onde total > 1000
- Busco o nome do cliente
- Ordeno do maior pro menor

In [ ]:
pipeline = [
    {
        "$match": {
            "total": {"$gt": 1000}
        }
    },
    {
        "$lookup": {
            "from": "clientes",
            "localField": "Id_Cliente",
            "foreignField": "_id",
            "as": "cliente"
        }
    },
    {
        "$unwind": "$cliente"
    },
    {
        "$project": {
            "_id": 0,
            "nome_cliente": "$cliente.nome",
            "data_pedido": "$dataPedido",
            "valor_total": "$total",
            "status": "$status",
            "num_itens": {"$size": "$itens"}
        }
    },
    {
        "$sort": {"valor_total": -1}
    }
]

resultado = list(bd.pedidos.aggregate(pipeline))
print("=== PEDIDOS COM VALOR ACIMA DE R$ 1000 ===")
if len(resultado) > 0:
    listTable(resultado)
else:
    print("Nenhum pedido encontrado com valor acima de R$ 1000.")
    print("\nMostrando todos os pedidos ordenados por valor:")
    # Mostrar todos os pedidos se não houver nenhum > 1000
    todos_pedidos = list(bd.pedidos.aggregate([
        {"$lookup": {"from": "clientes", "localField": "Id_Cliente", "foreignField": "_id", "as": "cliente"}},
        {"$unwind": "$cliente"},
        {"$project": {"_id": 0, "nome_cliente": "$cliente.nome", "valor_total": "$total", "status": "$status"}},
        {"$sort": {"valor_total": -1}}
    ]))
    listTable(todos_pedidos)

## A3.d) Top 10 produtos mais vendidos

Lista dos livros mais vendidos por quantidade.

**Como fiz:**
- Descompacto o array de itens dos pedidos
- Agrupo por produto e somo as quantidades
- Busco os dados do produto
- Ordeno do mais vendido pro menos vendido
- Pego só os 10 primeiros

In [ ]:
pipeline = [
    {
        "$unwind": "$itens"
    },
    {
        "$group": {
            "_id": "$itens.Id_Produto",
            "quantidade_vendida": {"$sum": "$itens.quantidade"}
        }
    },
    {
        "$lookup": {
            "from": "produtos",
            "localField": "_id",
            "foreignField": "_id",
            "as": "produto"
        }
    },
    {
        "$unwind": "$produto"
    },
    {
        "$project": {
            "_id": 0,
            "titulo": "$produto.titulo",
            "quantidade_vendida": 1,
            "preco_unitario": "$produto.preco",
            "receita_total": {"$multiply": ["$quantidade_vendida", "$produto.preco"]}
        }
    },
    {
        "$sort": {"quantidade_vendida": -1}
    },
    {
        "$limit": 10
    }
]

resultado = list(bd.pedidos.aggregate(pipeline))
print("=== TOP 10 PRODUTOS MAIS VENDIDOS ===")
listTable(resultado)

## A3.e) Clientes por região

Quantos clientes tem em cada região do Brasil.

**Como fiz:**
- Descompacto os endereços (já que um cliente pode ter vários)
- Agrupo por região
- Conto clientes únicos usando addToSet pra não repetir

In [ ]:
pipeline = [
    {
        "$unwind": "$endereco"
    },
    {
        "$group": {
            "_id": "$endereco.regiao",
            "clientes": {"$addToSet": "$_id"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "regiao": "$_id",
            "quantidade_clientes": {"$size": "$clientes"}
        }
    },
    {
        "$sort": {"quantidade_clientes": -1}
    }
]

resultado = list(bd.clientes.aggregate(pipeline))
print("=== QUANTIDADE DE CLIENTES POR REGIÃO ===")
listTable(resultado)

## CONSULTA ADICIONAL 1: Livros com múltiplos autores

Quero ver quais livros foram escritos por mais de um autor e quantos foram vendidos.

Achei interessante pra saber se livros com colaboração vendem bem.

In [ ]:
pipeline = [
    {
        "$match": {
            "$expr": {"$gt": [{"$size": "$autor"}, 1]}
        }
    },
    {
        "$lookup": {
            "from": "pedidos",
            "let": {"produto_id": "$_id"},
            "pipeline": [
                {"$unwind": "$itens"},
                {"$match": {"$expr": {"$eq": ["$itens.Id_Produto", "$$produto_id"]}}},
                {"$group": {"_id": None, "total_vendido": {"$sum": "$itens.quantidade"}}}
            ],
            "as": "vendas"
        }
    },
    {
        "$project": {
            "_id": 0,
            "titulo": 1,
            "autores": {
                "$reduce": {
                    "input": "$autor",
                    "initialValue": "",
                    "in": {
                        "$concat": [
                            "$$value",
                            {"$cond": [{"$eq": ["$$value", ""]}, "", ", "]},
                            "$$this.nome"
                        ]
                    }
                }
            },
            "num_autores": {"$size": "$autor"},
            "quantidade_vendida": {
                "$ifNull": [{"$arrayElemAt": ["$vendas.total_vendido", 0]}, 0]
            }
        }
    },
    {
        "$sort": {"quantidade_vendida": -1}
    }
]

resultado = list(bd.produtos.aggregate(pipeline))
print("=== LIVROS COM MÚLTIPLOS AUTORES E SUAS VENDAS ===")
listTable(resultado)

## CONSULTA ADICIONAL 2: Clientes fiéis

Identificar os clientes que fizeram mais de um pedido e quanto gastaram no total.

É útil pra saber quem são os melhores clientes e talvez fazer alguma promoção especial pra eles.

In [ ]:
pipeline = [
    {
        "$group": {
            "_id": "$Id_Cliente",
            "num_pedidos": {"$sum": 1},
            "valor_total_gasto": {"$sum": "$total"},
            "ticket_medio": {"$avg": "$total"},
            "data_primeiro_pedido": {"$min": "$dataPedido"},
            "data_ultimo_pedido": {"$max": "$dataPedido"}
        }
    },
    {
        "$match": {
            "num_pedidos": {"$gt": 1}
        }
    },
    {
        "$lookup": {
            "from": "clientes",
            "localField": "_id",
            "foreignField": "_id",
            "as": "cliente"
        }
    },
    {
        "$unwind": "$cliente"
    },
    {
        "$project": {
            "_id": 0,
            "nome_cliente": "$cliente.nome",
            "email": "$cliente.email",
            "num_pedidos": 1,
            "valor_total_gasto": {"$round": ["$valor_total_gasto", 2]},
            "ticket_medio": {"$round": ["$ticket_medio", 2]},
            "cliente_desde": "$data_primeiro_pedido",
            "ultimo_pedido": "$data_ultimo_pedido"
        }
    },
    {
        "$sort": {"valor_total_gasto": -1}
    }
]

resultado = list(bd.pedidos.aggregate(pipeline))
print("=== ANÁLISE DE CLIENTES FIÉIS (Múltiplos Pedidos) ===")
listTable(resultado)